In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError, NotFoundError
from elasticsearch.helpers import bulk
import json
import pandas as pd
import numpy as np
import string
import random
from datasets import load_dataset
#dataset = load_dataset("json", data_files="..\Preprocessor\output_preprocessing\corpus.json")

In [2]:
import json
with open("..\Preprocessor\output_preprocessing\corpus.json") as f:
    dataset = json.load(f)

In [3]:
dataset[3]['products'][0]['languages']['de']

[{'headerId': 1,
  'headerText': 'Ihr Beitrag zum Umweltschutz',
  'paragraphText': '',
  'headerChildren': [{'subHeaderId': 1,
    'subHeaderText': 'Entsorgung der Verpackung',
    'paragraphText': 'Die Verpackung dient der Handhabung und schützt das Gerät vor Transport- schäden. Die Verpackungsmaterialien sind nach umweltverträglichen und ent- sorgungstechnischen Gesichtspunkten ausgewählt und generell recycelbar. Das Rückführen der Verpackung in den Materialkreislauf spart Rohstoffe. Nut- zen Sie materialspezifische Wert- stoffsammlungen und Rückgabemög- lichkeiten. Wir empfehlen, die Verpackung für Transportzwecke aufzubewahren.'},
   {'subHeaderId': 2,
    'subHeaderText': 'Entsorgung des Altgerätes',
    'paragraphText': 'Elektro- und Elektronikgeräte enthalten vielfach wertvolle Materialien. Sie ent- halten auch bestimmte Stoffe, Gemi- sche und Bauteile, die für ihre Funktion und Sicherheit notwendig waren. Im Hausmüll sowie bei nicht sachgemäßer Behandlung können sie der mensch

In [5]:
def unique_id(size):
    chars = list(set(string.ascii_lowercase + string.digits).difference('LIO01'))
    return ''.join(random.choices(chars, k=size))


text = []
uid = []

for i in range(10):
    for j in range(len(dataset[i]['products'])):
        if "en" in dataset[i]['products'][j]['languages']:
            if len(dataset[i]['products'][j]['languages']['en']) != 0:
                for k in range(len(dataset[i]['products'][j]['languages']['en'])):
                    paragraphText = dataset[i]['products'][j]['languages']['en'][k]['paragraphText']
                    id = dataset[i]['manufacturer'] + '_' + unique_id(5)
                    text.append(paragraphText)
                    uid.append(id)
                    
list_ = [uid, text]
data = pd.DataFrame(list_).T
data.columns= ['uid', 'text']

In [6]:
from elastic import Elastic
elastic = Elastic()

In [7]:
elastic.create_index(name="coffee", config_file="./index_config.json")

[INFO] index coffee has been created!


In [33]:
from embedders import SwivelEmbedder, BertEmbedder
bert = BertEmbedder(bert_link="https://huggingface.co/nejox/bert-base-cased-squad2-coffee20230108")
swivel = SwivelEmbedder()

data['swivel_embedding'] = data['text'].apply(lambda t: swivel.embed_single_text(t)[0].tolist())
data['bert_embedding'] = data['text'].apply(lambda t: bert.embed_single_text(t)[0].tolist())

OSError: https://huggingface.co/nejox/bert-base-cased-squad2-coffee20230108 does not appear to be a valid module.

In [9]:
data

,uid,text,swivel_embedding,bert_embedding
0,AEG_kxtia,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0028328481130301952, 0.9882192015647888, -..."
1,AEG_2odzo,Press 2 sec to start descaling process Add Des...,"[0.3826048280323716, 0.02312913718205815, -0.1...","[0.05613254755735397, 0.9431564807891846, 0.81..."
2,AEG_7px93,After 100 cycles After 100 cycles OFF Wait for...,"[1.106480776675, 0.22196092774999993, -0.28723...","[0.03859584033489227, 0.9681762456893921, 0.81..."
3,AEG_zhy7v,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0028328481130301952, 0.9882192015647888, -..."
4,AEG_snqjc,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0028328481130301952, 0.9882192015647888, -..."
...,...,...,...,...
7508,Delonghi_e75tm,The light comes on after a period of time esta...,"[0.7748628916027285, 0.34861767962279994, -0.1...","[0.05727417394518852, 0.8437122702598572, 0.38..."
7509,Delonghi_4njbf,Voltage: 220-240 V~ 50/60 Hz max. 10 A Absorbe...,"[-0.13506272676822195, 0.6201836474429016, -0....","[-0.08802659064531326, -0.03395254164934158, -..."
7510,Delonghi_hhymu,In compliance with European directive 2002/96/...,"[0.4293976540421206, 0.6566117560765669, -0.47...","[-0.020026348531246185, 0.7846490144729614, 0...."
7511,Delonghi_o3avy,LIGHTS DESCRIPTION MEANING 1 8 9 2 3 4 5 6 7 1...,"[0.20701903465259813, -0.5565372581728936, 0.5...","[0.04705815389752388, 0.8337129950523376, 0.30..."


In [11]:
data['_op_type'] = "index"
data['_index'] = "coffee"
data_complete = data[['_op_type', '_index', 'uid', 'text', 'swivel_embedding', 'bert_embedding']]
index_data = data_complete.to_dict(orient='records')
print(len(index_data))
print(index_data[2])

7513
{'_op_type': 'index', '_index': 'coffee', 'uid': 'AEG_7px93', 'text': 'After 100 cycles After 100 cycles OFF Wait for the brewing time to ﬁnish Wait for the brewing time to ﬁnish Press alarm button for 2 sec Wait for the brewing time to ﬁnish * Always preheat the machine in case there are less than 4 cups or if the machine hasn’t been used in a long time. The machine is ready to use', 'swivel_embedding': [1.106480776675, 0.22196092774999993, -0.287237677875, 0.06950983056250003, -0.547103463, -0.7420425045375, 0.31376183725000006, 0.12530530003750004, -0.28588434277499997, -0.37947700468124995, 0.29845097616249994, 0.21036913915749997, -0.53307277850075, -0.20827327187500003, 0.02059105649999994, 0.9706959022124999, -0.7797185590875, 0.29512636648999996, 0.004186004049999987, -1.148225770875, -0.06113397475, -2.361816036875, -0.21739734025000002, -0.16135363362500002, 0.184736417975, -0.22279745085000002, -0.08917039475000009, -0.09388734757125003, -0.0969934342875, 0.056532940124

In [12]:
elastic.index_documents(index_data)

In [13]:
def retrieve(query:str, n_returns:int, index_name: str, embedder='bert'):
    '''
    return: 2-D list [['uid': cosine_similarity]], sorted by cosine score descendingly
    e.g.[['AEG_h3j5b', 0.963506],
         ['Melitta_u2k5s', 0.9492332000000001],
         ['Jura_q8g4h', 0.9438302999999999]]
    '''
    if embedder == 'swivel':
        query_vector = swivel.embed_single_text(query)[0].tolist()
    elif embedder == 'bert':
        query_vector = bert.embed_single_text(query)[0].tolist()
    else:
        raise ValueError("embedder must be either 'swivel' or 'bert'!")
        
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": f"cosineSimilarity(params.query_vector, '{embedder}_embedding') + 1.0", # Add 1.0 because ES does not support negative score
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    response = elastic.search(index_name, script_query, n_returns, list_fields_to_return=["uid"])
    
    res = []
    for hit in response["hits"]["hits"]:
        res.append([hit["_source"]["uid"], hit["_score"] - 1])
    return np.array(res)

In [31]:
query = "unplug the machine"

print("10 best matching documents of the query:")
results = retrieve(query=query, n_returns=10, index_name="coffee", embedder='bert')
winner = results[0][0]

10 best matching documents of the query:


In [32]:
data_complete[data_complete["uid"] == winner]["text"].iloc[0]

'n Use'